In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys

import time
import pandas as pd
from urllib.parse import quote

In [2]:
base_url = "https://www.rozee.pk/"
# search_query = "Software Engineer"
# search_url = f"https://www.rozee.pk/job/jsearch/q/{quote(search_query)}"

# print(search_url)

In [22]:


service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.get(url=base_url)



WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.NAME, "job_title"))
)

search_box = driver.find_element(By.NAME, "job_title")

# Step 4: Enter search query
search_box.send_keys("Software Engineer")

# Step 5: Press Enter
search_box.send_keys(Keys.RETURN)

WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.CLASS_NAME, "job "))
)

jobs = driver.find_elements(By.CLASS_NAME, "job ")  

Title = []
Company_Name = []
Location = []
Salary = []
Required_Skills = []
Job_Type = []
Job_Description = []
Job_Link = []




print(len(jobs))
# print(job.text for job in jobs)

main_window_handle = driver.current_window_handle

print(f'job:\n{jobs[5]}\n\n job\n {jobs[6]}')
i=0
while i < len(jobs):   
    
    try:
        if jobs[i].is_displayed() == False:
            # print(f'{i} is not displayed')
            i += 1
            continue 
    except:
        pass
        # print(f'{i} is neither available nor displayed')
        # i += 1
        # continue
    print(f'{i}, {driver.current_url}')
    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "jhead"))
        )
        jobhead = jobs[i].find_element(By.CLASS_NAME, "jhead")  

        print(f'jobhead: {jobhead}')

        title_element = jobhead.find_element(By.CLASS_NAME, "s-18")  # Find the <h3> element
        print(f'title_element: {title_element}')    
        title = title_element.get_attribute("title")  # Get the job title
        print(title)
        

        titleLink = title_element.find_element(By.TAG_NAME, "a").get_attribute("href")  # Get the job link
        print(titleLink)

        print(driver.current_url)

        driver.execute_script(f"window.open('{titleLink}', '_blank');")
        driver.switch_to.window(driver.window_handles[-1])

        print(driver.current_url)

        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'jHead')]"))
        )
        jhead = driver.find_element(By.XPATH, "//div[contains(@class, 'jHead')]")  
        jobCompany = jhead.find_element(By.XPATH, "//h2[contains(@class, 'cname')]").find_element(By.TAG_NAME, "a")  


        company_name = jobCompany.text
        company_link = jobCompany.get_attribute("href")

        print(company_name)
        print(company_link)


        company = jhead.find_element(By.XPATH, "//h4[contains(@class, 'lh1 cname')]")
        location = ''
        if company.text != '' or company.text != None:
            print('loc in if')
            location = company.text + ', ' + company.find_elements(By.TAG_NAME, "a")[-1].text
        else:
            print('loc in else')
            company = company.find_elements(By.TAG_NAME, "a")
            location += ', '.join([d.text for d in company])
            
        

        salary =  None
        try:
            salary = jhead.find_element(By.XPATH, "//div[contains(@class, 'mrsl')]").text.strip()
        except:
            salary = "Not Available"
        if salary == "" or salary == None or salary == " ":
            salary = "Not Available"
        print(location)
        print(salary)


        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")





        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'jobdetail')]"))
        )

        jobdetail = driver.find_element(By.XPATH, "//div[contains(@class, 'jobdetail')]")  

        jobdescdir = jobdetail.find_element(By.XPATH, ".//*[@dir='ltr']")
        
        jobdesc = jobdescdir.find_elements(By.TAG_NAME, "p")

        for p in jobdesc:
             if p.text == '':
                 continue
             description = p.text
             break

        print(f'description: {description}')

        jobrequirements_list = jobdetail.find_element(By.XPATH, "//div[contains(@class, 'jcnt')]").find_elements(By.TAG_NAME, "a") 
        job_requirements = '\n '.join([d.text for d in jobrequirements_list])

        print(job_requirements)

        

        description_list_rows = jobdetail.find_element(By.XPATH, "//div[contains(@class, 'jcnt jobd')]" )
        description_list_rows = description_list_rows.find_elements(By.CLASS_NAME, "row")
        job_type = ''
        for row in description_list_rows:
            if row.find_element(By.XPATH, "//div[contains(@class, 'col-lg-3')]").text == 'Job Type:':
                vals = row.find_element(By.XPATH, "//div[contains(@class, 'col-lg-7')]").find_elements(By.TAG_NAME, "a")
                job_type = '/ '.join([d.text for d in vals])

            


        print(job_type)

        
        Title.append(title)
        Company_Name.append(company_name)
        Location.append(location)
        Salary.append(salary)
        Required_Skills.append(job_requirements)
        Job_Type.append(job_type)
        Job_Description.append(description)
        Job_Link.append(titleLink)



        driver.close()
        driver.switch_to.window(main_window_handle)
        print(f'switched to main window {driver.current_url}')


        if i >= len(jobs) - 1 :
            pagination_element = driver.find_element(By.CLASS_NAME, "paginations")
            pagination_element.find_element(By.XPATH, "//a[contains(text(), 'Next')]").click()

            driver.switch_to.window(driver.window_handles[-1])
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

            main_window_handle = driver.current_window_handle

            print(f'pagination {driver.current_url}')

            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "job "))
            )

            new_jobs = driver.find_elements(By.CLASS_NAME, "job ")  
            print(f'new jobs: {len(new_jobs)}')
            jobs += new_jobs



        i += 1
    except Exception as e:
        # print(f"Error in {i}\n {driver.current_url}\n {e}")
        i+=1
    
    if i >= 29:
        break
    

data = pd.DataFrame({
    'Title': Title,
    'Company Name': Company_Name,
    'Location': Location,
    'Salary': Salary,
    'Required Skills': Required_Skills,
    'Job Type': Job_Type,
    'Job Description': Job_Description,
    'Job Link': Job_Link
})
    

driver.quit()


25
job:
<selenium.webdriver.remote.webelement.WebElement (session="e1d2eca6278371b6bfeb92cd68227199", element="f.F6E97E529B92815A102A36B14E88673B.d.2727C87FB3166FE17E0EF4F42E85A770.e.241")>

 job
 <selenium.webdriver.remote.webelement.WebElement (session="e1d2eca6278371b6bfeb92cd68227199", element="f.F6E97E529B92815A102A36B14E88673B.d.2727C87FB3166FE17E0EF4F42E85A770.e.242")>
5, https://www.rozee.pk/job/jsearch/q/Software%20Engineer
jobhead: <selenium.webdriver.remote.webelement.WebElement (session="e1d2eca6278371b6bfeb92cd68227199", element="f.F6E97E529B92815A102A36B14E88673B.d.2727C87FB3166FE17E0EF4F42E85A770.e.262")>
title_element: <selenium.webdriver.remote.webelement.WebElement (session="e1d2eca6278371b6bfeb92cd68227199", element="f.F6E97E529B92815A102A36B14E88673B.d.2727C87FB3166FE17E0EF4F42E85A770.e.263")>
Software Engineer
https://www.rozee.pk/croem-inc-software-engineer-islamabad-jobs-1575616?utm_source=jobSearch&utm_medium=website&utm_content=jobSearch_1575616&utm_campaign=RO

In [24]:
import pandas as pd
import re
import numpy as np

# Function to clean and parse salary values
def parse_salary(s):
    s = str(s).strip()
    if "not available" in s.lower() or not s:
        return np.nan  # Ignore jobs with no salary
    s = s.replace('PKR.', '').replace('/Month', '').replace(',', '').replace('+', '').strip()
    
    if '-' in s:  # Handle salary ranges
        parts = s.split('-')
        try:
            low = float(parts[0])
            high = float(parts[1])
            return (low + high) / 2.0
        except ValueError:
            return np.nan
    else:
        try:
            return float(s)
        except ValueError:
            return np.nan

# Convert the "Salary" column into numeric values
data['Salary'] = data['Salary'].apply(parse_salary)

# Ignore job postings with no salary information
data = data.dropna(subset=['Salary'])

# 1. Identify the most common job titles
title_counts = data['Title'].value_counts()
print("Most Common Job Titles:")
print(title_counts)

# 2. Calculate the average salary in Lahore (only for jobs with a given salary)
lahore_jobs = data[data['Location'].str.contains('Lahore', case=False, na=False)]
avg_salary_lahore = lahore_jobs['Salary'].mean()
print("\nAverage Salary in Lahore:", avg_salary_lahore)

# 3. Determine the most frequently required skills
all_skills = (
    data['Required Skills']
    .dropna()
    .apply(lambda x: re.split(r'[\n\t]+', x))  # Split on new lines or tabs
    .explode()
    .str.strip()
)

skill_counts = all_skills.value_counts()
print("\nMost Frequently Required Skills:")
print(skill_counts)


Most Common Job Titles:
Title
Software Quality Assurance Engineer    3
Software Engineer                      2
Senior Software Engineer               1
Software Support Engineer              1
Full Stack Software Engineer           1
Name: count, dtype: int64

Average Salary in Lahore: nan

Most Frequently Required Skills:
Required Skills
Performance Testing             3
Software Development            2
HTML                            2
C                               2
JavaScript                      2
ASP.net                         2
Software Design                 1
Regression Testing              1
Load Testing                    1
#MVC                            1
MS SQL                          1
#Git                            1
.Net                            1
MySQL Server                    1
T-SQL                           1
Jira Configuration              1
Software Analysis               1
Backend Development             1
jQuery                          1
Automation L

There are no jobs in lahore whose salaries are given, so cant take mean

In [11]:
data.to_csv('rozeepk.csv', index=False)